# BSC-HGP - Project  


| Course 	           | BSc                   |
|  ------------- :     |:-------------        :|  
| Stage / Year         | 3                     |
| Module 	           | HCI & GUI Programming | 
| Semester             |	1                  |
| Assignment Title     |	Project            | 
| Date of Title Issue  |	13th Nov 2018      |
| Assignment Deadline  |	14th Nov 2018 23:55|
| Assignment %         | 60 % of module        |
| Assignment Submission| Upload via Moodle     |

Author: Alex Cronin

# Submission by: Lisa Lewitanski - 2982190, Loic Doyen - 2981909


## Summary of Division of Work
* Lisa Lewitanski 2982190: 
Creation of the repository using Gitlab.  
Creation and display of the menus (Game, AI, View, Help and Exit) and the toolbar (Time Passed, Player's image, the turn, the player's timer, the remaining and the number of jumps).   
Creation and display of the gameboard (Display the gameboard using image for the white and black case, display the player's piece using image (white, black, white king and black king piece)).   
Display the winner of the game.   
Implementation of the winner detection when the user takes more than 5 minutes to play.   
Bonus:  
Implementation of an artificial intelligence (MinMax algorithm) which contains 3 levels of difficulty (easy, intermediate and expert). The artificial intelligence doesn't manage the king roles. If an AI can eat a piece of the opponent, it must eat it.   


* Loic Doyen 2981909:   
Implementation of the rules and the player's turn.   
Implementation of the king roles.    
Manage clicks event (to select two pieces).    
Implementation of the piece's deselection (by clicking again on the piece).   
Surrounded the piece that the player can player.  
Manage the resize of the screen.  
Manage the end of the game.  
Bonus:  
If the player can eat a piece of the opponent, it must eat it.  


### Code repository 
![title](./img/gitlab1.png)
![title](./img/gitlab2.png)
![title](./img/gitlab3.png)

### Percentage of work completed by each partner on each class / task 


| Class name/Task 	| Lisa Lewitanski 	| Loic Doyen 	|
|--------------------------------	|----------------	|----------------	|
| Draughts 	        |100%  	| 0%	|
| Board 	        | 20% 	|  80%	|
| ScoreBoard        | 90% | 10% |
| AI                | 90%   |   10% |
| System Design 	 |  50%	|  	50%|
| Git hub repository 	| 50% 	| 50% 	|
| Learning the rules of Draughts 	| 50% 	| 50% 	|
| Writing the rules in logic  	|  50%	|  50%	|


 


## Solution (tasks 1 to 9)
Place your solution in the cell(s) below
Include any error message or any special comments in this cell

In [ ]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QApplication, QAction, QFileDialog, QWidget,\
    QVBoxLayout, QLCDNumber, QSlider, QMessageBox, QLabel, QDockWidget, QListWidget, QTextEdit
from PyQt5.QtWidgets import *
from PyQt5.QtCore import Qt, QBasicTimer, pyqtSignal, QPoint, QTimer, QTime, QRect, QDate
from PyQt5.QtGui import QPainter, QColor, QImage, QIcon, QPen, QPixmap, QLinearGradient, QPainterPath, QtGui, QtCore
import sys
import os
import sip
import time

PATH_BLACK_PIECE = "./icons/black_piece_panel.png"
PATH_WHITE_PIECE = "./icons/white_piece_panel.png"

WINNER_P1 = "The winner is: player 1 !"
WINNER_P2 = "The winner is: player 2 !"
WINNER_AI = "The winner is: the AI !"
CLEAR_IMAGE = "./icons/clear.png"
PAUSE_IMAGE = "./icons/pause.png"
STATUS_IMAGE = "./icons/status.png"
HELP_IMAGE = "./icons/help.png"
EXIT_IMAGE = "./icons/logout.png"
ROBOT_EASY = "./icons/roboteasy.png"
ROBOT_MEDIUM = "./icons/robotmedium.png"
ROBOT_HARD = "./icons/robothard.png"
WHITE_PIECE = "./icons/white_piece.png"
BLACK_PIECE = "./icons/black_piece2.png"
WHITE_KING = "./icons/white_king.png"
BLACK_KING = "./icons/black_king.png"
WHITE_SQUARE = "./icons/white_square.png"
BLACK_SQUARE = "./icons/black_square.png"
FPLAYER = "Waiting first player to play"
SPLAYER = "Waiting second player to play"
AIPLAYER = "Waiting AI player to play"
ENDGAME = "The game is over"

if __name__ == '__main__':
    app = QApplication([])
    draughts = Draughts()
    sys.exit(app.exec_())


class Board(QFrame):
    """
    Everything to handle the game: Drawing pieces and the board, Handling the mouse and the game logic
    """
    msg2Statusbar = pyqtSignal(str)

    # todo set the board with and height in square
    boardWidth = 8
    boardHeight = 8
    Speed = 300

    def __init__(self, parent):
        super().__init__(parent)
        self.init()

    def init(self):
        """
        Initialise all values

        :return:
        """
        self.boardArray = [[0, 3, 0, 3, 0, 3, 0, 3],
                           [3, 0, 3, 0, 3, 0, 3, 0],
                           [0, 3, 0, 3, 0, 3, 0, 3],
                           [1, 0, 1, 0, 1, 0, 1, 0],
                           [0, 1, 0, 1, 0, 1, 0, 1],
                           [2, 0, 2, 0, 2, 0, 2, 0],
                           [0, 2, 0, 2, 0, 2, 0, 2],
                           [2, 0, 2, 0, 2, 0, 2, 0]]
        # Timer initialization

        self.timer = QTimer()
        self.time = QTime(0, 0, 0)
        self.timer.timeout.connect(self.timerEventGame)

        self.timerPlayer = QTimer()
        self.timePlayer = QTime(0, 1, 0)
        self.timer.timeout.connect(self.timerEventPlayer)

        self.isWaitingAfterLine = False
        self.numLinesRemoved = 0
        # image de la piece de base m'voyez
        self.image = QImage(WHITE_PIECE)
        # self.setFocusPolicy(Qt.StrongFocus)
        self.isStarted = False
        self.isPaused = False
        self.reset_game()
        self.selected_piece = [-1, -1]
        self.player_turn = 1
        # nombre de piece = [nb de piece joueur 1, nb de piece jour 2]
        self.playersRemaining = [12, 12]
        # nombre de jump = [nb de jump joueur 1, nb de jump jour 2]
        self.playersJumps = [0, 0]
        self.scoreBoard = None
        self.status = FPLAYER
        # si le temps du joueur a timeout
        self.timePassed = False
        self.startTime = 0
        self.interval = 0
        # pour changer l'affichage si on jou contre une AI
        self.isAI = False
        self.winner = -1
        self.aiDifficulties = 2

    def timerEventGame(self):
        """
        This function allows to update the game timer and update the scoreBoard.

        :return: Nothing
        """
        # Update the game timer
        self.time = self.time.addSecs(1)
        # Update de UI of the scoreBoard
        self.scoreBoard.updateUI()

    def initPlayerTimer(self):
        """
        Init the player timer when the player begin his turn.

        :return: Nothing
        """
        self.timerPlayer = QTimer()
        self.timePlayer = QTime(0, 1, 0)

    def startTimerInter(self, interval):
        """
        Start the player timer with an given interval.

        :param interval: The interval from which time must resume
        :return: Nothing
        """
        self.timerPlayer.start(interval)

    def startTimerPlayer(self):
        """"
        Function to start a player timer with an interval of 1000.
        """
        self.interval = 1000
        self.startTime = time.time()
        self.timerPlayer.start(1000)

    def timerEventPlayer(self):
        """
        This function allows to update the timer of the player one.

        :return: Nothing
        """
        self.timePlayer = self.timePlayer.addSecs(-1)
        if self.timerPlayer.isActive():
            if self.timePlayer.minute() * 60 + self.timePlayer.second() < 1:
                self.timePassed = True
                self.timerPassedEndGame()
        self.scoreBoard.updateUI()

    def setScoreBoard(self, scoreBoard):
        """
        This function allows to get the scoreBoard class to use it later in the code.

        :param scoreBoard: The class scoreBoard
        :return: Nothing
        """
        self.scoreBoard = scoreBoard

    def print_board_array(self):
        """
        Prints the boardArray in an attractive way

        :return: Nothing
        """

        print("boardArray:")
        print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in self.boardArray]))

    def mouse_pos_to_col_row(self, event):
        """
        Convert the mouse click event to a row and column.

        :param event: The click event
        :return: The click's position with the following format: [x, y] (Example: [3, 4])
        """

        return [int(event.x() / self.square_width()), int(event.y() / self.square_height())]

    def square_width(self):
        """
        This function allows to return the width of one square in the board.

        :return: The width of one square in the board.
        """
        return self.contentsRect().width() / Board.boardWidth

    def square_height(self):
        """
        This function allows to return the height of one square in the board.

        :return: The height of one square in the board
        """

        return self.contentsRect().height() / Board.boardHeight

    def start(self):
        """starts game"""
        if self.isPaused:
            return

        self.isStarted = True
        self.isWaitingAfterLine = False
        self.numLinesRemoved = 0
        self.reset_game()
        self.timer.start(1000)
        self.startTimerPlayer()
        # self.timerPlayer.start(1000)

    def pause(self):
        """pauses game"""

        if not self.isStarted:
            return

        self.isPaused = not self.isPaused

        if self.isPaused:
            self.timerPlayer.stop()
            self.timer.stop()
            self.status = "Game paused"
            elapsedTime = self.startTime - time.time()
            self.startTime -= elapsedTime
            self.interval -= int(elapsedTime * 1000)

        else:
            self.timer.start()
            self.startTimerInter(self.interval)
            if self.player_turn == 1:
                self.status = FPLAYER
            elif self.isAI:
                self.status = AIPLAYER
            else:
                self.status = SPLAYER
            self.scoreBoard.updateUI()

        self.update()

    def paintEvent(self, event):
        """paints the board and the pieces of the game"""
        painter = QPainter(self)
        self.draw_board_squares(painter)
        self.draw_pieces(painter)

    def new_place(self, turn, row, col, change):
        """
        Change the piece place

        :param turn:
        :param row:
        :param col:
        :param change:
        :return:
        """
        self.boardArray[row][col] = self.boardArray[self.selected_piece[1]][self.selected_piece[0]]
        if (row == 0 and self.boardArray[row][col] == 2) or (row == 7 and self.boardArray[row][col] == 3):
            self.boardArray[row][col] += 2
        self.boardArray[self.selected_piece[1]][self.selected_piece[0]] = 1
        if change == 1:
            self.selected_piece = [-1, -1]
        else:
            self.selected_piece = [col, row]
        self.player_turn = turn
        self.scoreBoard.updateUI()

    def first_player_take(self, row, col):
        """
        Take a piece for the first player

        :param row:
        :param col:
        :return:
        """
        p = 2
        change = 1
        b = 0
        r = self.selected_piece[1]
        c = self.selected_piece[0]
        if self.boardArray[r][c] == 2 and r > 1 and c > 1 and self.boardArray[r - 2][c - 2] == 1 and \
                        self.boardArray[r - 1][c - 1] == 3:
            b = 1
        elif self.boardArray[r][c] == 2 and r > 1 and c < 6 and self.boardArray[r - 2][c + 2] == 1 and \
                        self.boardArray[r - 1][c + 1] == 3:
            b = 1
        if b == 0 and self.player_turn == 1 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] - 1 \
                and (col == self.selected_piece[0] + 1 or col == self.selected_piece[0] - 1):
            self.new_place(2, row, col, change)
        elif self.player_turn == 1 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] - 2 \
                and col == self.selected_piece[0] + 2 and self.boardArray[row + 1][col - 1] == 3:
            self.boardArray[row + 1][col - 1] = 1
            self.playersRemaining[1] -= 1
            if self.playersRemaining[1] == 0:
                self.timePassed = True
                self.player_turn = 2
                self.timerPassedEndGame()
            self.playersJumps[0] += 1
            if row > 1 and col > 1 and self.boardArray[row - 2][col - 2] == 1 and self.boardArray[row - 1][col - 1] == 3:
                p = 1
                change = 0
            elif row > 1 and col < 6 and self.boardArray[row - 2][col + 2] == 1 and self.boardArray[row - 1][col + 1] == 3:
                p = 1
                change = 0
            self.new_place(p, row, col, change)
        elif self.player_turn == 1 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] - 2 \
                and col == self.selected_piece[0] - 2 and self.boardArray[row + 1][col + 1] == 3:
            self.boardArray[row + 1][col + 1] = 1
            self.playersRemaining[1] -= 1
            if self.playersRemaining[1] == 0:
                self.timePassed = True
                self.player_turn = 2
                self.timerPassedEndGame()
            self.playersJumps[0] += 1
            if row > 1 and col > 1 and self.boardArray[row - 2][col - 2] == 1 and self.boardArray[row - 1][col - 1] == 3:
                p = 1
                change = 0
            elif row > 1 and col < 6 and self.boardArray[row - 2][col + 2] == 1 and self.boardArray[row - 1][col + 1] == 3:
                p = 1
                change = 0
            self.new_place(p, row, col, change)

    def second_player_take(self, row, col):
        """
        Take a piece for the second player

        :param row:
        :param col:
        :return:
        """
        p = 1
        change = 1
        b = 0
        r = self.selected_piece[1]
        c = self.selected_piece[0]
        if self.boardArray[r][c] == 3 and r < 6 and c > 1 and self.boardArray[r + 2][c - 2] == 1 and \
                        self.boardArray[r + 1][c - 1] == 2:
            b = 1
        elif self.boardArray[r][c] == 3 and r < 6 and c < 6 and self.boardArray[r + 2][c + 2] == 1 and \
                        self.boardArray[r + 1][c + 1] == 2:
            b = 1
        if b == 0 and self.player_turn == 2 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] + 1 \
                and (col == self.selected_piece[0] + 1 or col == self.selected_piece[0] - 1):
            self.new_place(p, row, col, change)
        elif self.player_turn == 2 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] + 2 \
                and col == self.selected_piece[0] + 2 and self.boardArray[row - 1][col - 1] == 2:
            self.boardArray[row - 1][col - 1] = 1
            self.playersRemaining[0] -= 1
            if self.playersRemaining[0] == 0:
                self.timePassed = True
                self.player_turn = 1
                self.timerPassedEndGame()
            self.playersJumps[1] += 1
            if row < 6 and col > 1 and self.boardArray[row + 2][col - 2] == 1 and self.boardArray[row + 1][col - 1] == 2:
                p = 2
                change = 0
            elif row < 6 and col < 6 and self.boardArray[row + 2][col + 2] == 1 and self.boardArray[row + 1][col + 1] == 2:
                p = 2
                change = 0
            self.new_place(p, row, col, change)
        elif self.player_turn == 2 and self.boardArray[row][col] == 1 and row == self.selected_piece[1] + 2 \
                and col == self.selected_piece[0] - 2 and self.boardArray[row - 1][col + 1] == 2:
            self.boardArray[row - 1][col + 1] = 1
            self.playersRemaining[0] -= 1
            if self.playersRemaining[0] == 0:
                self.timePassed = True
                self.player_turn = 1
                self.timerPassedEndGame()
            self.playersJumps[1] += 1
            if row < 6 and col > 1 and self.boardArray[row + 2][col - 2] == 1 and self.boardArray[row + 1][col - 1] == 2:
                p = 2
                change = 0
            elif row < 6 and col < 6 and self.boardArray[row + 2][col + 2] == 1 and self.boardArray[row + 1][col + 1] == 2:
                p = 2
                change = 0
            self.new_place(p, row, col, change)

    def mousePressEvent(self, event):
        """
        Get the pos clicked and call the game logic
        :param event:
        :return:
        """
        if not self.isPaused:
            clicked = self.mouse_pos_to_col_row(event)
            if clicked:
                self.logicGame(clicked)

    def AI(self):
        """
        Creates the AI class with the array board.
        Then, it get the positions choosed by the AI.
        Finally it call the logic Game to move the piece.

        :return: Nothing
        """

        self.selected_piece = [-1, -1]
        self.ai = AI(self.boardArray)
        positions = self.ai.MinMaxDecision(self.aiDifficulties)
        self.logicGame([positions[0][1], positions[0][0]])
        self.logicGame([positions[1][1], positions[1][0]])

    def timerPassedEndGame(self):
        """
        When a player take a lot of time to play (1 minutes)

        :return: -1 (value for the end of the game)
        """
        self.status = ENDGAME
        self.timerPlayer.stop()
        self.timer.stop()
        if self.player_turn == 1:
            if not self.isAI:
                self.winner = 2
            else:
                self.winner = 3
        else:
            self.winner = 1
        # self.scoreBoard.updateUI()
        # self.init()
        return -1

    def logicGame(self, positions):
        """
        Game logic to manage pieces moves
        :param positions:
        :return:
        """
        row = positions[1]
        col = positions[0]
        if self.timePassed:
            return self.timerPassedEndGame()
        if self.player_turn == 1:
            self.status = FPLAYER
        elif self.isAI:
            self.status = AIPLAYER
        else:
            self.status = SPLAYER

        if self.selected_piece == [-1, -1]:
            if self.player_turn == 1:
                b = 0
                for r in range(0, len(self.boardArray)):
                    for c in range(0, len(self.boardArray[0])):
                        if self.boardArray[r][c] == 2 and r > 1 and c > 1 and self.boardArray[r - 2][c - 2] == 1 and self.boardArray[r - 1][c - 1] == 3:
                            b = 1
                        elif self.boardArray[r][c] == 2 and r > 1 and c < 6 and self.boardArray[r - 2][c + 2] == 1 and self.boardArray[r - 1][c + 1] == 3:
                            b = 1
                if self.boardArray[row][col] == self.player_turn + 1:
                    if (b == 0 and row > 0 and col > 0 and self.boardArray[row - 1][col - 1] == 1) \
                            or (b == 0 and row > 0 and col < 7 and self.boardArray[row - 1][col + 1] == 1):
                        self.selected_piece = positions
                    elif row > 1 and col > 1 and self.boardArray[row - 2][col - 2] == 1 and self.boardArray[row - 1][col - 1] == 3:
                        self.selected_piece = positions
                    elif row > 1 and col < 6 and self.boardArray[row - 2][col + 2] == 1 and self.boardArray[row - 1][col + 1] == 3:
                        self.selected_piece = positions
            elif self.player_turn == 2:
                b = 0
                for r in range(0, len(self.boardArray)):
                    for c in range(0, len(self.boardArray[0])):
                        if self.boardArray[r][c] == 3 and r < 6 and c > 1 and self.boardArray[r + 2][c - 2] == 1 and self.boardArray[r + 1][c - 1] == 2:
                            b = 1
                        elif self.boardArray[r][c] == 3 and r < 6 and c < 6 and self.boardArray[r + 2][c + 2] == 1 and self.boardArray[r + 1][c + 1] == 2:
                            b = 1
                if self.boardArray[row][col] == self.player_turn + 1:
                    if (b == 0 and row < 7 and col > 0 and self.boardArray[row + 1][col - 1] == 1) \
                        or (b == 0 and row < 7 and col < 7 and self.boardArray[row + 1][col + 1] == 1):
                        self.selected_piece = positions
                    elif row < 6 and col > 1 and self.boardArray[row + 2][col - 2] == 1 and self.boardArray[row + 1][col - 1] == 2:
                        self.selected_piece = positions
                    elif row < 6 and col < 6 and self.boardArray[row + 2][col + 2] == 1 and self.boardArray[row + 1][col + 1] == 2:
                        self.selected_piece = positions

        else:
            if self.selected_piece == positions:
                self.selected_piece = [-1,-1]
                return
            self.first_player_take(row, col)
            self.second_player_take(row, col)
            if self.player_turn == 1:
                self.status = FPLAYER
            else:
                self.status = SPLAYER
            self.initPlayerTimer()
            self.startTimerPlayer()
            if self.player_turn == 2 and self.isAI:
                self.AI()

    def keyPressEvent(self, event):
        """processes key press events if you would like to do any"""
        if not self.isStarted or self.curPiece.shape() == Piece.NoPiece:
            super(Board, self).keyPressEvent(event)
            return
        key = event.key()
        if key == Qt.Key_P:
            self.pause()
            return
        if self.isPaused:
            return
        elif key == Qt.Key_Left:
            self.try_move(self.curPiece, self.curX - 1, self.curY)
        elif key == Qt.Key_Right:
            self.try_move(self.curPiece, self.curX + 1, self.curY)
        elif key == Qt.Key_Down:
            self.try_move(self.curPiece.rotateRight(), self.curX, self.curY)
        elif key == Qt.Key_Up:
            self.try_move(self.curPiece.rotateLeft(), self.curX, self.curY)
        elif key == Qt.Key_Space:
            self.dropDown()
        elif key == Qt.Key_D:
            self.oneLineDown()
        else:
            super(Board, self).keyPressEvent(event)

    def timerEvent(self, event):
        """handles timer event"""

        #todo adapter this code to handle your timers
        if event.timerId() == self.timer.timerId():
            pass
        else:
            super(Board, self).timerEvent(event)

    def reset_game(self):
        """clears pieces from the board"""

        # 2d int/Piece array to story the state of the game
        # 2 pion blanc, 3 pion noir
        self.boardArray = [[0, 3, 0, 3, 0, 3, 0, 3],
                           [3, 0, 3, 0, 3, 0, 3, 0],
                           [0, 3, 0, 3, 0, 3, 0, 3],
                           [1, 0, 1, 0, 1, 0, 1, 0],
                           [0, 1, 0, 1, 0, 1, 0, 1],
                           [2, 0, 2, 0, 2, 0, 2, 0],
                           [0, 2, 0, 2, 0, 2, 0, 2],
                           [2, 0, 2, 0, 2, 0, 2, 0]]
        self.selected_piece = [-1, -1]
        # self.print_board_array()

    def try_move(self, new_x, new_y):
        """tries to move a piece"""

    def draw_board_squares(self, painter):
        """
        This function allows to draw all the square on the board.

        :param painter: The painter
        :return: Nothing
        """

        # todo set the default colour of the brush
        images = [QImage(WHITE_SQUARE), QImage(BLACK_SQUARE)]
        idx = 0
        for row in range(0, Board.boardHeight):
            for col in range(0, Board.boardWidth):
                #painter.save()
                # Todo set this value equal the transformation you would like in the column direction (x)
                colTransformation = col * self.square_width()

                # Todo set this value equal the transformation you would like in the column direction (y)
                row_transformation = row * self.square_height()

                final_image = images[idx].scaled(self.square_width(), self.square_height())
                painter.drawImage(colTransformation, row_transformation, final_image)
                #painter.restore()
                idx = 1 if idx == 0 else 0
            idx = 1 if idx == 0 else 0

    def color_brush_white(self, row, col, i):
        """
        Set the brush color for white pieces
        :param row:
        :param col:
        :param i:
        :return:
        """
        brush_color = Qt.black
        if self.player_turn == 1 and self.selected_piece[1] == row and self.selected_piece[0] == col:
            brush_color = QColor.fromRgb(100, 255, 100)
        elif self.player_turn == 1 and self.selected_piece == [-1, -1]:
            if i > 0 and row > 0 and col > 0 and self.boardArray[row - 1][col - 1] == 1:
                brush_color = Qt.white
            elif i > 0 and row > 0 and col < 7 and self.boardArray[row - 1][col + 1] == 1:
                brush_color = Qt.white
            elif row > 1 and col > 1 and self.boardArray[row - 2][col - 2] == 1 and self.boardArray[row - 1][col - 1] == 3:
                brush_color = Qt.white
            elif row > 1 and col < 6 and self.boardArray[row - 2][col + 2] == 1 and self.boardArray[row - 1][col + 1] == 3:
                brush_color = Qt.white
        return brush_color

    def color_brush_black(self, row, col, i):
        """
        Set the brush color for black pieces
        :param row:
        :param col:
        :param i:
        :return:
        """
        brush_color = Qt.black
        if self.player_turn == 2 and self.selected_piece[1] == row and self.selected_piece[0] == col:
            brush_color = QColor.fromRgb(100, 100, 255)
        elif self.player_turn == 2 and self.selected_piece == [-1, -1]:
            if i > 0 and row < 7 and col > 0 and self.boardArray[row + 1][col - 1] == 1:
                brush_color = QColor.fromRgb(255, 0, 255)
            elif i > 0 and row < 7 and col < 7 and self.boardArray[row + 1][col + 1] == 1:
                brush_color = QColor.fromRgb(255, 0, 255)
            elif row < 6 and col > 1 and self.boardArray[row + 2][col - 2] == 1 and self.boardArray[row + 1][col - 1] == 2:
                brush_color = QColor.fromRgb(255, 0, 255)
            elif row < 6 and col < 6 and self.boardArray[row + 2][col + 2] == 1 and self.boardArray[row + 1][col + 1] == 2:
                brush_color = QColor.fromRgb(255, 0, 255)
        return brush_color

    def draw_pieces(self, painter):
        """draw the prices on the board"""
        brush_color = QColor.fromRgb(200, 200, 200)
        images = [QImage(WHITE_PIECE), QImage(BLACK_PIECE), QImage(WHITE_KING), QImage(BLACK_KING), QImage(BLACK_SQUARE)]
        idx = 0
        i = 0
        y = 1
        while i < 2:
            for row in range(0, len(self.boardArray)):
                for col in range(0, len(self.boardArray[0])):
                    #painter.save()
                    col_transformation = col * self.square_width()
                    row_transformation = row * self.square_height()
                    ## Todo victoire quand bloquer

                    if self.boardArray[row][col] == 2:
                        brush_color = self.color_brush_white(row, col, i)
                        idx = 0
                    elif self.boardArray[row][col] == 1:
                        brush_color = Qt.black
                        idx = 4
                    elif self.boardArray[row][col] == 3:
                        brush_color = self.color_brush_black(row, col, i)
                        idx = 1
                    elif self.boardArray[row][col] == 4:
                        brush_color = self.color_brush_white(row, col, i)
                        idx = 2
                    elif self.boardArray[row][col] == 5:
                        brush_color = self.color_brush_black(row, col, i)
                        idx = 3
                    elif self.boardArray[row][col] == 6:
                        brush_color = QColor.fromRgb(128, 128, 128)
                    if self.boardArray[row][col] != 0:
                        painter.setPen(brush_color)
                        if brush_color == Qt.white or brush_color == QColor.fromRgb(255, 0, 255):
                            y = 2
                        # Todo draw some the pieces as eclipses
                        radius_width = (self.square_width() / 10 * 8) / 2
                        radius_height = (self.square_height() / 10 * 8) / 2
                        center = QPoint(col_transformation + (self.square_width() / 2),
                                        row_transformation + (self.square_height() / 2))
                        if idx != 4:
                            painter.drawEllipse(center, radius_width, radius_height)
                            self.image = images[idx].scaled(radius_width * 2, radius_height * 2)
                            painter.drawImage(center.x() - radius_width, center.y() - radius_height, self.image)
                        #painter.restore()
                        self.update()
            i += y


class Draughts(QMainWindow):
    """
    This class allows to create the board for the game and the scoreboard to display all information.
    """

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        """
        Initilization of the game board and the score board.

        :return: Nothing
        """
        # Initialization of the board class
        self.tboard = Board(self)
        self.setCentralWidget(self.tboard)

        # Initialization of the ScoreBoard class
        self.scoreBoard = ScoreBoard(self.tboard)
        self.tboard.setScoreBoard(self.scoreBoard)

        # Set the score board in a dockwidget at the right of the window
        self.addDockWidget(Qt.RightDockWidgetArea, self.scoreBoard)

        # Initialization of the status bar
        self.statusbar = self.statusBar()
        self.tboard.msg2Statusbar[str].connect(self.statusbar.showMessage)

        # Start the class Board
        self.tboard.start()
        self.gameToolbar()

        # Initialization of the windows
        self.resize(900, 800)
        self.setFixedSize(self.size())
        self.center()
        self.setWindowTitle('Checkers')
        self.show()

    def center(self):
        """
        Centers the window on the screen
        :return: Nothing
        """

        screen = QDesktopWidget().screenGeometry()
        size = self.geometry()
        self.move((screen.width() - size.width()) / 2,
                  (screen.height() - size.height()) / 2)

    def gameToolbar(self):
        """
        This function allows to create the Menubar.
        It contains th Game, the View, the Help and the Exit menu.
        :return: Nothing
        """
        # Declaration of each menu in the menu bar.
        mainMenu = self.menuBar()
        self.resetMenu = mainMenu.addMenu(" Game")
        self.aiMenu = mainMenu.addMenu(" AI")
        self.viewMenu = mainMenu.addMenu(" View")
        self.helpMenu = mainMenu.addMenu(" Help")
        self.exitMenu = mainMenu.addMenu(" Exit")

        # Initialization of each menu in the menu bar.
        self.gameSettings()
        self.gameAIEasy()
        self.gameAIMedium()
        self.gameAIHard()
        self.pauseGame()
        self.viewSettings()
        self.helpSettings()
        self.exitSettings()

    def gameSettings(self):
        """
        This function allows to reset the game.
        The shortcut is the following: Ctrl+R
        If the user click on this menu, the resetGame funtion will be call.

        :return: Nothing
        """
        resetAction = QAction(QIcon(CLEAR_IMAGE), "Reset", self)
        resetAction.setShortcut("Ctrl+R")
        self.resetMenu.addAction(resetAction)
        resetAction.triggered.connect(self.resetGame)

    def gameAIEasy(self):
        """
        This function allows to create a game against an artificial intelligence.
        The shortcut is the following: Ctrl+A
        If the user click on this menu, the AIPlayer funtion will be call.

        :return: Nothing
        """
        aiAction = QAction(QIcon(ROBOT_EASY), "Play against an easy AI", self)
        aiAction.setShortcut("Ctrl+A")
        self.aiMenu.addAction(aiAction)
        aiAction.triggered.connect(self.AIPlayerE)

    def gameAIMedium(self):
        """
        This function allows to create a game against an artificial intelligence.
        The shortcut is the following: Ctrl+A
        If the user click on this menu, the AIPlayer funtion will be call.

        :return: Nothing
        """
        aiAction = QAction(QIcon(ROBOT_MEDIUM), "Play against an medium AI", self)
        aiAction.setShortcut("Ctrl+Q")
        self.aiMenu.addAction(aiAction)
        aiAction.triggered.connect(self.AIPlayerM)

    def gameAIHard(self):
        """
        This function allows to create a game against an artificial intelligence.
        The shortcut is the following: Ctrl+A
        If the user click on this menu, the AIPlayer funtion will be call.

        :return: Nothing
        """
        aiAction = QAction(QIcon(ROBOT_HARD), "Play against an hard AI", self)
        aiAction.setShortcut("Ctrl+W")
        self.aiMenu.addAction(aiAction)
        aiAction.triggered.connect(self.AIPlayerH)

    def pauseGame(self):
        """
        This function allows set the game in pause.
        The shortcut is the following: Ctrl+P
        If the user click on this menu, the pause funtion will be call.

        :return: Nothing
        """
        resetAction = QAction(QIcon(PAUSE_IMAGE), "Pause / Continue", self)
        resetAction.setShortcut("Ctrl+P")
        self.resetMenu.addAction(resetAction)
        resetAction.triggered.connect(self.pause)

    def viewSettings(self):
        """
        This function allows to view the status of the game.
        The shortcut is the following: Ctrl+V
        If the user click on this menu, the viewStatus funtion will be call.

        :return: Nothing
        """

        resetAction = QAction(QIcon(STATUS_IMAGE), "View status", self)
        resetAction.setShortcut("Ctrl+V")
        self.viewMenu.addAction(resetAction)
        resetAction.triggered.connect(self.viewStatus)

    def helpSettings(self):
        """
        This function allows to display the rules of the game.
        The shortcut is the following: Ctrl+H
        If the user click on this menu, the help funtion will be call.

        :return: Nothing
        """

        helpAction = QAction(QIcon(HELP_IMAGE), "Help", self)
        helpAction.setShortcut("Ctrl+H")
        self.helpMenu.addAction(helpAction)
        helpAction.triggered.connect(self.help)

    def exitSettings(self):
        """
        This function allows to exit the checkers game.
        The shortcut is the following: Ctrl+E

        :return: Nothing
        """

        exitAction = QAction(QIcon(EXIT_IMAGE), "Exit", self)
        exitAction.setShortcut("Ctrl+E")
        self.exitMenu.addAction(exitAction)
        exitAction.triggered.connect(qApp.quit)

    def resetGame(self):
        """
        This function will initialize the game and the score board.

        :return: Nothing
        """
        # Initialization of the game
        self.tboard.init()
        self.tboard.start()
        # Initialization of the scoreBoard
        self.scoreBoard.setAI(False)
        self.scoreBoard.updateUI()
        self.tboard.setScoreBoard(self.scoreBoard)

    def pause(self):
        """
        This function will call the pause function in the Board class to set the game in pause.

        :return: Nothing
        """
        self.tboard.pause()

    def AIPlayerE(self):
        """
        Function to allows the player to play against an easy AI

        :return: Nothing
        """
        self.tboard.isAI = True
        self.scoreBoard.setAI(True)
        self.tboard.aiDifficulties = 1

    def AIPlayerM(self):
        """
        Function to allows the player to play against an medium AI

        :return: Nothing
        """
        self.tboard.isAI = True
        self.scoreBoard.setAI(True)
        self.tboard.aiDifficulties = 2

    def AIPlayerH(self):
        """
        Function to allows the player to play against an hard AI

        :return: Nothing
        """
        self.tboard.isAI = True
        self.scoreBoard.setAI(True)
        self.tboard.aiDifficulties = 3

    def viewStatus(self):
        """
        This function allows to display the status of the game in a QMessageBox.

        :return: Nothing
        """
        title = "Game status "
        message = self.tboard.status
        QMessageBox.about(self, title, message)
        self.show()

    def help(self):
        """
        This function display a QMessageBox. This box help the user to know what the application
        exactyly doing.
        Indeed, it contains rules of the game.

        :return: Nothing
        """
        title = "Help"
        message = "It's a checker game.\n" \
                  "The rules:\n" \
                  "Moves are allowed only on the dark squares, so pieces always move diagonally.\n" \
                  "Single pieces are always limited to forward moves.\n" \
                  "A piece making a move only on one square. A piece making a capturing move leaps over one of the opponent's pieces,\n" \
                  "landing in a straight diagonal line on the other side.\n" \
                  "Only one piece may be captured in a single jump; however, multiple jumps are allowed during a single turn.\n" \
                  "When a piece is captured, it is removed from the board.\n" \
                  "If a player is able to make a capture, there is no option; the jump must be made.\n" \
                  "If more than one capture is available, the player is free to choose whichever he or she prefers.\n" \
                  "When a piece reaches the furthest row from the player who controls that piece, it is crowned and becomes a king.\n" \
                  "One of the pieces which had been captured is placed on top of the king so that it is twice as high as a single piece.\n" \
                  "Kings are limited to moving diagonally but may move both forward and backward.\n" \
                  "Kings may combine jumps in several directions, forward and backward, on the same turn.\n" \
                  "Single pieces may shift direction diagonally during a multiple capture turn, but must always jump forward.\n"

        QMessageBox.about(self, title, message)
        self.show()

class ScoreBoard(QDockWidget):
    """
    This class allows to display the scoreboard.

    """
    def __init__(self, Board):
        super().__init__()
        self.initUI(Board)

    def initUI(self, Board):
        """
        Initialization of the ScoreBoard UI.

        :param Board: The class board which manages the game.
        :return: Nothing
        """

        # Set if the player play against an AI
        self.ai = False

        # Initalization of the window
        self.resize(200, 800)
        self.setFixedSize(self.size())
        self.center()
        self.setWindowTitle('Game Panel')

        # Initialization of the Board class
        self.boardInfos = Board

        # Initialization of the QWidget which contain all information
        self.dockedWidget = QWidget(self)
        self.setWidget(self.dockedWidget)

        # Initialization of the timer and the players information
        self.timer = self.display_timer()
        self.infosPlayer1 = self.displayInfoPlayer1()
        self.infosPlayer2 = self.displayInfoPlayer2()

        # Set all information in the widget
        self.dockedWidget.setLayout(QVBoxLayout())
        self.dockedWidget.layout().addWidget(self.timer)
        self.dockedWidget.layout().addWidget(self.infosPlayer1)
        self.dockedWidget.layout().addWidget(self.infosPlayer2)

    def setAI(self, isAI):
        """
        Set the boolean to true to modify the player's name

        :return: Nothing
        """
        self.ai = isAI

    def updateUI(self):
        """
        Update the UI to display the timer and when the user want to create a new game.

        :return: Nothing
        """
        # deletion of the timer widget with the timer's child
        self.dockedWidget.layout().removeWidget(self.timer)
        sip.delete(self.timer)
        self.timer = None

        # deletion of the player one widget with the players one's child
        self.dockedWidget.layout().removeWidget(self.infosPlayer1)
        sip.delete(self.infosPlayer1)
        self.infosPlayer1 = None

        # deletion of the player two widget with the player two's child
        self.dockedWidget.layout().removeWidget(self.infosPlayer2)
        sip.delete(self.infosPlayer2)
        self.infosPlayer2 = None

        # Initialization of the timer and the player's information
        self.timer = self.display_timer()
        self.infosPlayer1 = self.displayInfoPlayer1()
        self.infosPlayer2 = self.displayInfoPlayer2()

        # Set all information in the widget
        self.dockedWidget.layout().addWidget(self.timer)
        self.dockedWidget.layout().addWidget(self.infosPlayer1)
        self.dockedWidget.layout().addWidget(self.infosPlayer2)

    def displayInfoPlayer1(self):
        """

        This function allows to create the widget which contains the player one information.
        It displays the player name, the turn, the timer, the number of remaining and jumps.

        :return: playerOne: The widget which contains all information of the user
        """
        # Initialization of the player number
        name_player = 2
        # Initialization of player name
        if self.ai:
            player = QLabel("AI")
        else:
            player = QLabel("Player")

        # Initialization of the layout
        grid = QGridLayout()

        # Set all information's player in the layout
        grid.addWidget(self.get_image_player(name_player), 1, 1, 1, 2)
        grid.addWidget(self.get_turn(name_player), 1, 2, 1, 3)
        grid.addWidget(player, 2, 1)
        grid.addWidget(self.get_player_timer(name_player), 3, 1)
        grid.addWidget(self.get_remaining_label(name_player), 4, 1)
        grid.addWidget(self.get_jumps_label(name_player), 5, 1)

        # Initialization of the widget
        playerOne = QWidget(self)

        # Set the player layout in the widget
        playerOne.setLayout(grid)
        return playerOne

    def displayInfoPlayer2(self):
        """

        This function allows to create the widget which contains the player two information.
        It displays the player name, the turn, the timer, the number of remaining and jumps.

        :return: playerTwo: The widget which contains all information of the user
        """
        # Initialization of the player number
        name_player = 1

        # Initialization of player name
        if self.ai:
            player = QLabel("Player")
        else:
            player = QLabel("Opponent")

        # Initialization of the layout
        grid = QGridLayout()

        # Set all information's player in the layout
        grid.addWidget(self.get_image_player(name_player), 1, 1, 1, 2)
        grid.addWidget(self.get_turn(name_player), 1, 2, 1, 3)
        grid.addWidget(player, 2, 1)
        grid.addWidget(self.get_player_timer(name_player), 3, 1)
        grid.addWidget(self.get_remaining_label(name_player), 4, 1)
        grid.addWidget(self.get_jumps_label(name_player), 5, 1)

        # Initialization of the widget
        playerTwo = QWidget(self)

        # Set the player layout in the widget
        playerTwo.setLayout(grid)
        return playerTwo

    def get_player_timer(self, namePlayer):
        """
        This function allows to know the player's turn and display his timer if it's his turn.
        The format is the following: 'Timer: 00:00:00'

        :param namePlayer: The player number.
        :return: The Qlabel which contains the player's timer.
        """
        if self.boardInfos.player_turn == 1 and namePlayer == 1:
            timeP = "Timer: " + self.boardInfos.timePlayer.toString("mm:ss")
        elif self.boardInfos.player_turn == 2 and namePlayer == 2:
            timeP = "Timer: " + self.boardInfos.timePlayer.toString("mm:ss")
        else:
            timeP = "Timer: Stopped"
        return QLabel(timeP)

    def get_image_player(self, playerName):
        """
        This function allows to display the player's image.


        :param playerName: Player number
        :return: The image of the player (QPixmap)
        """
        image = QLabel()
        if playerName == 1:
            pixmap = QtGui.QPixmap(PATH_WHITE_PIECE)
        else:
            pixmap = QtGui.QPixmap(PATH_BLACK_PIECE)
        image.setPixmap(pixmap)
        return image

    def get_turn(self, namePlayer):
        """
        This function allows to get the player's turn and return a QLabel which contains 'Your turn' beside
        his image if it's his turn.
        If not the function return an empty QLabel.

        :param namePlayer: The player number
        :return: The QLabel which contains the player turn.
        """
        if self.boardInfos.player_turn == 1 and namePlayer == 1:
            return QLabel(" Your Turn")
        elif self.boardInfos.player_turn == 2 and namePlayer == 2:
            return QLabel(" Your Turn")
        else:
            return QLabel("")

    def get_remaining_label(self, numPlayer):
        """
        This function allows to get the player's ramaining and return it in a QLabel.
        The sentence have this format:  'Remaining: 12'.

        :param numPlayer: The player number .
        :return: QLabel which contains the player's remaining.
        """
        if numPlayer == 1:
            numRemaining = self.boardInfos.playersRemaining[0]
        else:
            numRemaining = self.boardInfos.playersRemaining[1]
        strRemaining = "Remaining: " + str(numRemaining)
        return QLabel(strRemaining)

    def get_jumps_label(self, numPlayer):
        """
        This function allows to get the player's jumps and return it in a QLabel.
        The sentence have this format:  'Jumps: 12'.

        :param numPlayer: The player number .
        :return: QLabel which contains the player's jumps.
        """
        if numPlayer == 1:
            numJumps = self.boardInfos.playersJumps[0]
        if numPlayer == 2:
            numJumps = self.boardInfos.playersJumps[1]
        strJump = "Jumps: " + str(numJumps)
        return QLabel(strJump)

    def display_timer(self):
        """
        This function allows to display the timer of the name of the winner.

        If the game is not over it get the timer and return it in to a QLabel.
        The format is the following: 'Time Passed: 00:00'

        If the game is  over it get the winner's name and return it in to a QLabel.

        :return: The QLabel which contains the timer or the winner of the game.
        """

        if self.boardInfos.timePassed:
            if self.boardInfos.winner == 1:
                winnerGame = WINNER_P1
            elif self.boardInfos.winner == 2:
                winnerGame = WINNER_P2
            elif self.boardInfos.winner == 3:
                winnerGame = WINNER_AI
            return QLabel(winnerGame)
        else:
            gameTime = "Time Passed: " + self.boardInfos.time.toString("mm:ss")
            return QLabel(gameTime)

    def center(self):
        '''centers the window on the screen'''
        pass

class AI():
    """
    This method allows the AI to choose his move.
    This AI can't manage the  king's role.

    """
    def __init__(self, board):
        super().__init__()
        self.replay = False
        # copy of the game board
        self.gameBoard = [row[:] for row in board]
        # Increase the recursion's limit
        limit = sys.getrecursionlimit()
        sys.setrecursionlimit(3500)

    def Simulation(self, selected_piece, row, col, isMax, depth, board):
        """
        Remove piece if a player eat it.
        Call the MinMaxValue function to get the value according to the simulation of the shot.

        :param selected_piece: The piece position
        :param row: The piece position in row with the move
        :param col: The piece position in col with the move
        :return: The result of the MinMaxValue funtion

        """
        tmp_board = [row[:] for row in self.gameBoard]
        maxRow = len(tmp_board)
        maxCol = len(tmp_board[maxRow - 1])
        if isMax:
            tmp_board[row][col] = 3
        else:
            tmp_board[row][col] = 2
        replay = False

        # Remove the white piece on the board if the AI eat it
        if isMax and (row + 1 < maxRow and col + 1 >= 0 and row == selected_piece[0] + 2 and col == selected_piece[1] + 2):
            tmp_board[row + 1][col + 1] = 1
        elif isMax and (row + 1 < maxRow and col - 1 >= 0 and row == selected_piece[0] + 2 and col == selected_piece[1] - 2):
            tmp_board[row + 1][col - 1] = 1
        # Remove the black piece on the board if the Player eat it
        elif not isMax and (row - 1 < maxRow and col + 1 >= 0 and row == selected_piece[0] - 2 and col == selected_piece[1] + 2):
            tmp_board[row - 1][col + 1] = 1
        elif not isMax and (row - 1 < maxRow and col - 1 >= 0 and row == selected_piece[0] - 2 and col == selected_piece[1] - 2):
            tmp_board[row - 1][col - 1] = 1

        # Check if a piece become a king
        if isMax and row == len(tmp_board) - 1:
            tmp_board[row][col] = 5
        elif not isMax and row == 0:
            tmp_board[row][col] = 4

        tmp_board[selected_piece[0]][selected_piece[1]] = 1

        self.replay = replay
        return self.MinMaxValue(depth, isMax, tmp_board)

    def PieceCanMove(self, tmp_board, isMax):
        """
        Get each move that the player can do.

        :return: Return a list containig each move that the player can do.
        """
        maxRow = len(tmp_board)
        maxCol = len(tmp_board[maxRow - 1])
        piece_move = []

        for row in range(0, maxRow):
            for col in range(0, maxCol):
                if (isMax and tmp_board[row][col] == 3) or (not isMax and tmp_board[row][col] == 2):
                    # Save all move that the black player can do in the piece_move list
                    if isMax and (row + 1 < maxRow and col - 1 >= 0 and tmp_board[row + 1][col - 1] == 1) or \
                            (row + 1 < maxRow and col + 1 < maxCol and tmp_board[row + 1][col + 1] == 1) or \
                            (row + 2 < maxRow and col - 2 >= 0 and tmp_board[row + 2][col - 2] == 1 and
                             tmp_board[row + 1][col - 1] == 2) or \
                            (row + 2 < maxRow and col + 2 < maxCol and tmp_board[row + 2][col + 2] == 1 and
                             tmp_board[row + 1][col + 1] == 2):
                        piece_move.append([row, col])

                    # Save all move that the white player can do in the piece_move list
                    if not isMax and (row - 1 >= 0 and col - 1 >= 0 and tmp_board[row - 1][col - 1] == 1) or \
                            (row - 1 >= 0 and col + 1 < maxCol and tmp_board[row - 1][col + 1] == 1) or \
                            (row - 2 >= 0 and col - 2 >= 0 and tmp_board[row - 2][col - 2] == 1 and
                             tmp_board[row - 1][col - 1] == 3) or \
                            (row - 2 >= 0 and col + 2 < maxCol and tmp_board[row - 2][col + 2] == 1 and
                             tmp_board[row - 1][col + 1] == 3):
                        piece_move.append([row, col])
        return piece_move

    def MinMaxDecision(self, depth):
        """
        Show each shot, the player can do.
        Call the MinMaxValue function to create the "shot tree".

        :param depth: The level.
        :return: Return a list containing the x and y position that the player will do.
        """
        maxRow = len(self.gameBoard)
        maxCol = len(self.gameBoard[maxRow - 1])
        selected = []
        destination = []
        value = []

        for row in range(0, maxRow):
            for col in range(0, maxCol):
                if self.gameBoard[row][col] == 3:
                    # if the player can move at the bottom left
                    if row + 1 < maxRow and col - 1 >= 0 and self.gameBoard[row + 1][col - 1] == 1:
                        selected.append([row, col])
                        destination.append([row + 1, col - 1])
                        value.append(self.Simulation([row, col], row + 1, col - 1, False, depth, self.gameBoard))

                    # if the player can move at the bottom right
                    if row + 1 < maxRow and col + 1 < maxCol and self.gameBoard[row + 1][col + 1] == 1:
                        selected.append([row, col])
                        destination.append([row + 1, col + 1])
                        value.append(self.Simulation([row, col], row + 1, col + 1, False, depth, self.gameBoard))

                    # if the player can move at the bottom left to eat the opponent
                    if row + 2 < maxRow and col - 2 >= 0 and self.gameBoard[row + 2][col - 2] == 1 and \
                            self.gameBoard[row + 1][col - 1] == 2:
                        selected.append([row, col])
                        return [[row, col], [row + 2, col - 2]]

                    # if the player can move at the bottom right to eat the opponent
                    if row + 2 < maxRow and col + 2 < maxCol and self.gameBoard[row + 2][col + 2] == 1 and \
                            self.gameBoard[row + 1][col + 1] == 2:
                        selected.append([row, col])
                        destination.append([row + 2, col + 2])
                        return [[row, col], [row + 2, col + 2]]

        tmp_val = value[0]
        idx = 0
        for count in range(0, len(value)):
            if value[count] > tmp_val:
                tmp_val = value[count]
                idx = count
        return [selected[idx], destination[idx]]

    def isWinner(self, tmp_board, numPlayer):
        """
        Show if the player win.

        :param tmp_board: The board.
        :param numPlayer: The player
        :return: True  if the player win, else it return false.
        """
        for row in tmp_board:
            for col in row:
                if col == numPlayer:
                    return False
        return True

    def Evaluation(self, board):
        """
        Return the value of the shot.

        :return: value that corresponds to the importance of the move.
        """
        # 1 for a normal piece, 1.5 for a king
        black, white = 0, 0
        for row in board:
            for cell in row:
                if cell == 3:
                    black += 1.0
                elif cell == 5:
                    black += 1.5
                elif cell == 2:
                    white += 1.0
                elif cell == 4:
                    white += 1.5
        return black - white

    def MinMaxValue(self, depth, isMax, tmp_board):
        """

        Get the best move according to the player.

        :param depth: The depth
        :param isMax: True if it's the player black, else false
        :param tmp_board: The board

        :return: The value of the best move
        """

        maxRow = len(tmp_board)
        maxCol = len(tmp_board[maxRow - 1])

        # If the player win
        if self.isWinner(tmp_board, 3):
            return 100
        # If the AI win
        if self.isWinner(tmp_board, 2):
            return -100

        # if there are no more depth
        if depth == 0:
            return self.Evaluation(tmp_board)

        vals = []
        piece_can_move = self.PieceCanMove(tmp_board, isMax)

        for piece in piece_can_move:
            # Simulation of the AI move
            # if the AI can move at the bottom left
            if isMax and piece[0] + 1 < maxRow and piece[1] - 1 >= 0 and tmp_board[piece[0] + 1][piece[1] - 1] == 1:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] + 1, piece[1] - 1, not isMax, depth - 1, tmp_board))
                # if the AI can move at the bottom right
            if isMax and piece[0] + 1 < maxRow and piece[1] + 1 < maxCol and tmp_board[piece[0] + 1][piece[1] + 1] == 1:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] + 1, piece[1] + 1, not isMax, depth - 1, tmp_board))
            # if the AI can move at the bottom left to eat the opponent
            if isMax and piece[0] + 2 < maxRow and piece[1] - 2 >= 0 and tmp_board[piece[0] + 2][piece[1] - 2] == 1 and \
                    tmp_board[piece[0] + 1][piece[1] - 1] == 2:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] + 2, piece[1] - 2, not isMax, depth - 1, tmp_board))
                # if the AI can move at the bottom right to eat the opponent
            if isMax and piece[0] + 2 < maxRow and piece[1] + 2 < maxCol and tmp_board[piece[0] + 2][piece[1] + 2] == 1 and \
                    tmp_board[piece[0] + 1][piece[1] + 1] == 2:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] + 2, piece[1] + 2, not isMax, depth - 1, tmp_board))

            # Simulation of the Player move
                # if the player can move at the top left
            if not isMax and piece[0] - 1 >= 0 and piece[1] - 1 >= 0 and tmp_board[piece[0] - 1][piece[1] - 1] == 1:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] - 1, piece[1] - 1, not isMax, depth - 1, tmp_board))
                # if the player can move at the top right
            if not isMax and piece[0] - 1 >= 0 and piece[1] + 1 < maxCol and tmp_board[piece[0] - 1][piece[1] + 1] == 1:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] - 1, piece[1] + 1, not isMax, depth - 1, tmp_board))
                # if the player can move at the top left to eat the opponent
            if not isMax and piece[0] - 2 >= 0 and piece[1] - 2 >= 0 and tmp_board[piece[0] - 2][piece[1] - 2] == 1 and \
                    tmp_board[piece[0] - 1][piece[1] - 1] == 3:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] - 2, piece[1] - 2, not isMax, depth - 1, tmp_board))
                # if the player can move at the top right to eat the opponent
            if not isMax and piece[0] - 2 >= 0 and piece[1] + 2 < maxCol and tmp_board[piece[0] - 2][piece[1] + 2] == 1 and \
                    tmp_board[piece[0] - 1][piece[1] + 1] == 3:
                vals.append(self.Simulation([piece[0], piece[1]], piece[0] - 2, piece[1] + 2, not isMax, depth - 1, tmp_board))

        retVal = 0
        if len(vals) > 0:
            retVal = vals[0]
        for elem in vals:
            if isMax and elem > retVal:
                retVal = elem
            elif not isMax and elem < retVal:
                retVal = elem
        return retVal



## Design Decisions:  (task 10) 

#### Toolbar
First of all we have created an toolbar which contains the Game, AI, View, Help and Exit menus at the top of the screen.    
In the Game menu the user could reset or pause/continue the game.    
In the AI menu the user can choose the difficulty of the artificial intelligence(AI).  
In the View menu the user can click on the view status submenu to show the status of the game (if the game is on pause, the name of the winner etc).  
Then there are the menu Help. With this menu, the user could read the rules of the game.  
To finish there are the Exit menu to leave the application.   
Each submenu contains an icon and a shortcut. This functionnality help the user to be more fast and to get a better understanding of the usefulness of each submenu.   

#### ScoreBoard
Then there are the score board at the rigth of the screen because is the commun way to display this information. The Score board is a docketwidget which means that the player can move the window of place.  
The score board contains the time passed and the information about each players (image, turn, timer, remaining and jumps).   
The background of the score board is grey to show easely each information.      
At the end of the game, the time passed is replace by the name of the winner. (Player, Opponent or AI)  
When the AI mode is activated, the name of the black player becomes 'AI' and the status is update. When it will be the turn of the AI, in the status, the name of the black player will be also 'AI'.      
When an information is modified, all information are automatically update.  
The information of the player one and the player two are separate to avoid confusion.  

#### GameBoard

The first player will be always the white player, like in the real rules.  
The opponent or the AI will be always the black player.  
The game board looks a lot like a classic wooden board made of light beige and dark brown box.  
The piece white and black are realistics. Indeed they are images of classic checkers piece.  
It gives the impression to the player to play on a real board.  
If the player want to play against an AI, it must launch a new game and select the level of the AI.  
By example, after a party it could launch a game against an AI with this following shortcup: Ctrl+R and Ctrl+A.  
When it will the turn of a player, the piece that it can move becomes surrrounded by a color. The color to the white player is green and the color for the black player is pink.  
If a player can eat a piece of the opponent, it must eat it. So when a piece must eat, it's the only one surrouded by his color. We have chosen this color because we see them clearly.    
The player can deselect a piece by clicking again on it.  

To conclude the board is classic and we did everything for the players to have the best visualization of the board.  
The application is full react that which help the user to know everything about the game.  





## Screen Shots of working Project x 5
![title](./img/begin_game.png)
![title](./img/in_game.png)
![title](./img/in_game2.png)
![title](./img/score_board.png)
![title](./img/end_game.png)